In [1]:
import pandas as pd
import numpy as np
import ipyleaflet
import datetime
from ipyleaflet import Map, basemaps, basemap_to_tiles, Marker, CircleMarker, MarkerCluster, AwesomeIcon, FullScreenControl
from datetime import datetime as dt
from ipywidgets import HTML

In [2]:
df = pd.read_csv('../data/SensorFloods.csv')               #Sensors with floods labelled
df['Start_time'] = pd.to_datetime(df['Start_time'])
df['Start_time'] = df['Start_time'].dt.date

In [3]:
dfAll = pd.read_csv('../data/AllEventsData.csv')             #All Sensors with any data labelled
dfAll['Start_time'] = pd.to_datetime(dfAll['Start_time'])
dfAll['Start_time'] = dfAll['Start_time'].dt.date

In [4]:
df311 = pd.read_csv('../data/311ServiceRequests.csv', low_memory=False)      #311 Complaints for Street Flooding
#convert created date column to datetime values
df311['Created Date'] = pd.to_datetime(df311['Created Date'])
#filter out years before 2020
df311= df311[df311['Created Date'].dt.year > 2019]

df311 = df311.drop(columns=['Agency Name','Location Type', 'Vehicle Type','Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment', 'Open Data Channel Type',
       'Park Facility Name'])
df311['Created Date'] = df311['Created Date'].dt.date
df311['Created Date'] = pd.to_datetime(df311['Created Date'])


### Mapping Sensors and 311 data within a certain time period

In [5]:
def map(start=None, end=None):
    #Format input date
    start = dt.strptime(start,'%Y-%m-%d')
    styear = start.year
    stmonth = start.month
    stday = start.day

    end = dt.strptime(end,'%Y-%m-%d')
    endyear = end.year
    endmonth = end.month
    endday = end.day

    #Create Map
    center = (40.6542974,-73.8316683)
    m = Map(basemap=basemaps.CartoDB.DarkMatter,center=center, zoom=12)
    
    #Create icons 
    icon1 = AwesomeIcon(name = 'circle', marker_color = 'blue', icon_color= 'white',spin = False)
    icon2 = AwesomeIcon(name = 'circle',marker_color = 'white',icon_color= 'black',spin = False)


    #Plot all sensors with flood data 
    new = df.drop_duplicates('Deployment_id')

    for index, row in new.iterrows():
        lat = new['Latitude'][index]
        long = new['Longitude'][index]
        marker = Marker(icon=icon1,location=(lat,long),draggable= False)
        m.add_layer(marker)
        address = HTML()
        address.value = new['Address'][index]
        marker.popup = address

    #Mapping all deployed sensors with no recorded floods
    floodsensors = ['easily_dear_mouse','duly_pro_vervet','mainly_fond_boar','mildly_calm_ibex',
                'daily_happy_satyr','simply_pet_joey','daily_new_falcon','weekly_poetic_guinea']

    newALL = dfAll.drop_duplicates('Deployment_id')

    for index, row in newALL.iterrows():
        if newALL['Deployment_id'][index] not in floodsensors:
            lat = newALL['Latitude'][index]
            long = newALL['Longitude'][index]
            marker = Marker(icon=icon2,location=(lat,long),draggable= False)
            m.add_layer(marker)
            address = HTML()
            address.value = newALL['Address'][index]
            marker.popup = address
        
        
    
    #Mapping all 311 Flood Complaints since input date
    
    df311['Created Date'] = pd.to_datetime(df311['Created Date'])

    df311['year'] = df311['Created Date'].dt.year
    df311['month'] = df311['Created Date'].dt.month
    df311['day'] = df311['Created Date'].dt.day

    df311['Created Date'] = df311['Created Date'].astype(str)

    for index,row in df311.iterrows():
        date = dt.strptime(df311['Created Date'][index],'%Y-%m-%d')
        if start <= date <= end:
            lat = df311['Latitude'][index]
            long = df311['Longitude'][index]
            circle_marker = CircleMarker()
            circle_marker.location = (lat,long)
            circle_marker.radius = 2
            circle_marker.color = 'red'
            m.add_layer(circle_marker)

            df311['Created Date'] = df311['Created Date'].astype(str)
            dateVal = HTML()
            dateVal.value = df311['Created Date'][index]
            circle_marker.popup = dateVal
    
    return m

In [6]:
map(start='2021-12-30', end = '2022-1-6')

C:\Users\sadhv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\jupyter_client\session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Map(center=[40.6542974, -73.8316683], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…